In [1]:
%pwd

'd:\\dpl-project\\Kidney-CNN-DP-Learning\\research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'd:\\dpl-project\\Kidney-CNN-DP-Learning'

### entity

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    """Data Ingestion Configuration"""
    root_dir: Path
    source_URl: str
    local_data_file: Path
    unzip_dir: Path

##### configuration

In [15]:
from src.CNN_Classification.constants import *
from src.CNN_Classification.utils.common import read_yaml , create_directories
import os
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        if not os.path.exists(config_filepath):
            raise FileNotFoundError(f"Configuration file not found: {config_filepath}")
        if not os.path.exists(params_filepath):
            raise FileNotFoundError(f"Parameters file not found: {params_filepath}")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URl=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

##### components

In [16]:
import os
import zipfile
from src.CNN_Classification.utils.common import get_size
import gdown
from src.CNN_Classification import logger


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

        
    def download_file(self)-> str:
        try:
            dataset_url = self.config.source_URl
            zip_download_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion',exist_ok=True)

            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)
           
            logger.info(f"Download data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

#### pipeline

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise e

[2025-04-05 14:43:48,942: INFO: common: yaml file:config\config.yaml loaded successfully]
[2025-04-05 14:43:48,942: INFO: common: yaml file:params.yaml loaded successfully]
[2025-04-05 14:43:48,942: INFO: common: created directory at: artifacts]
[2025-04-05 14:43:48,957: INFO: common: created directory at: artifacts/data_ingestion]
[2025-04-05 14:43:48,959: INFO: 4218785560: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=64bc812f-96f5-4ce1-8fb2-4cb600fc66f0
To: d:\dpl-project\Kidney-CNN-DP-Learning\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [01:05<00:00, 879kB/s] 

[2025-04-05 14:44:59,859: INFO: 4218785560: Download data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
